In [13]:
from PIL import Image, ImageDraw, ImageFont, ImageOps, ExifTags
import os
import piexif
from datetime import datetime

# For generating watermarks

def correct_orientation(image):
    """
    Correct the orientation of an image using its EXIF data.
    """
    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == 'Orientation':
            break

    exif = image._getexif()

    if exif is not None and orientation in exif:
        if exif[orientation] == 3:
            image = image.rotate(180, expand=True)
        elif exif[orientation] == 6:
            image = image.rotate(270, expand=True)
        elif exif[orientation] == 8:
            image = image.rotate(90, expand=True)

    return image

def add_watermark(image_path, text, save_as=None):
    original = Image.open(image_path)
    original = correct_orientation(original)
    txt = Image.new('RGBA', original.size, (255,255,255,0))
    draw = ImageDraw.Draw(txt)

    # Scale the font size based on the image size
    base_width = 800  # Change this to your preferred base width
    font_size = int((min(original.size) / base_width) * 20)
    font_size = max(font_size, 10)  # Set a minimum font size if needed

    font = ImageFont.truetype("arial.ttf", font_size)
    text_x = original.width // 2
    text_y = original.height - 30

    text_position = (text_x, text_y)
    draw.text(text_position, text, fill=(255,255,255,128), font=font, anchor="mm")
    watermarked = Image.alpha_composite(original.convert('RGBA'), txt).convert('RGB')
    
    if save_as:
        watermarked.save(save_as, 'JPEG')
    else:
        watermarked.save(image_path, 'JPEG')

def get_camera_info(exif_data):
    camera_info = {}
    # Only add EXIF data to the dictionary if it is not the default 'Unknown'
    if piexif.ImageIFD.Model in exif_data['0th']:
        camera_info['model'] = exif_data['0th'][piexif.ImageIFD.Model].decode('utf-8')

    if piexif.ExifIFD.ISOSpeedRatings in exif_data['Exif']:
        camera_info['iso'] = str(exif_data['Exif'][piexif.ExifIFD.ISOSpeedRatings])

    if piexif.ExifIFD.FNumber in exif_data['Exif']:
        f_number = exif_data['Exif'][piexif.ExifIFD.FNumber]
        camera_info['fstop'] = f"{f_number[0] / f_number[1]}"

    if piexif.ExifIFD.ExposureTime in exif_data['Exif']:
        exposure_time = exif_data['Exif'][piexif.ExifIFD.ExposureTime]
        camera_info['shutterspeed'] = f"{exposure_time[0]}/{exposure_time[1]} sec"

    if piexif.ExifIFD.DateTimeOriginal in exif_data['Exif']:
        try:
            camera_info['datetime'] = exif_data['Exif'][piexif.ExifIFD.DateTimeOriginal].decode('utf-8')
            camera_info['year'] = datetime.strptime(camera_info['datetime'], '%Y:%m:%d %H:%M:%S').year
        except:
            pass

    return camera_info

def process_images(source_folder_path):
    target_folder_path = '../images/watermark'
    os.makedirs(target_folder_path, exist_ok=True)

    for filename in os.listdir(source_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(source_folder_path, filename)
            exif_data = piexif.load(image_path)
            camera_info = get_camera_info(exif_data)

            # Construct watermark text
            watermark_elements = [str(camera_info.get('year', ''))]  # Start with the year
            if 'model' in camera_info:
                watermark_elements.append(camera_info['model'])
            if 'iso' in camera_info:
                watermark_elements.append(f"ISO {camera_info['iso']}")
            if 'shutterspeed' in camera_info:
                watermark_elements.append(camera_info['shutterspeed'])

            # Remove empty strings and join with ', '
            watermark_text = "(c) Charles Lehnen " + ', '.join(filter(None, watermark_elements))

            save_as_path = os.path.join(target_folder_path, f"{os.path.splitext(filename)[0]}_watermarked.jpg")
            add_watermark(image_path, watermark_text, save_as=save_as_path)

process_images('../images/no_watermark')


In [14]:
import os
from PIL import Image

# For generating list of files to copy/paste into photos.qmd

# Directory where the images are stored
image_folder = '../images/watermark/'
# Prefix for the path to use in the markdown output
output_path_prefix = 'images/watermark/'

# Categorize images into portrait and landscape
landscape_images = []
portrait_images = []

for filename in os.listdir(image_folder):
    if filename.lower().endswith('.jpg'):
        with Image.open(os.path.join(image_folder, filename)) as img:
            if img.width > img.height:
                landscape_images.append(filename)
            else:
                portrait_images.append(filename)

# Combine the lists, placing landscape images first
images = landscape_images + portrait_images

# Generate markdown output
markdown_output = "::: {layout=\"[[1, 1], [1, 1]]\"}\n"
for image in images:
    image_path = os.path.join(output_path_prefix, image).replace("\\", "/")
    markdown_output += f"![]({image_path}){{group=\"my-gallery\"}}\n\n"
markdown_output += ":::"

print(markdown_output)





::: {layout="[[1, 1], [1, 1]]"}
![](images/watermark/amblyrhynchus_cristatus_watermarked.jpg){group="my-gallery"}

![](images/watermark/california_sea_lion_watermarked.jpg){group="my-gallery"}

![](images/watermark/conolophus_subcristalus_watermarked.jpg){group="my-gallery"}

![](images/watermark/iguana_iguana_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_3895_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5437_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5444_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5670_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5868_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_7778_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3015_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3232_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3254_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3255_watermar

In [10]:
import pandas as pd

try:
    projects_df = pd.read_csv('../documents/projects.csv')
except UnicodeDecodeError:
    # If a UnicodeDecodeError is encountered, try reading with 'latin1' encoding
    projects_df = pd.read_csv('../documents/projects.csv', encoding='latin1')

# Generate the server function script for the Shiny app using the DataFrame

# Template for a single marker in the server function
marker_template = """
addMarkers(
  lng = {lng}, 
  lat = {lat}, 
  popup = "{popup}", 
  icon = {icon_name}Icon
)
"""

# Initialize the base of the server function string
server_function_script = "server <- function(input, output, session) {\n"

# Generate the makeIcon function calls for each unique icon
icon_template = """
  {icon_name}Icon <- makeIcon(
    iconUrl = "../images/icons/{icon_file}",
    iconWidth = {icon_width}, iconHeight = {icon_height},
    iconAnchorX = 25, iconAnchorY = 50
  )
"""

# Assuming icons are 50x50 by default, you can add conditional logic for different sizes if necessary
for icon_file in projects_df['Icon'].unique():
    icon_name = icon_file.split('.')[0]  # Assuming the file name before the extension is the icon name
    server_function_script += icon_template.format(
        icon_name=icon_name,
        icon_file=icon_file,
        icon_width=50,  # Default width
        icon_height=50  # Default height
    )

# Add the leaflet rendering function call
server_function_script += "\n  output$map <- renderLeaflet({\n    leaflet() %>%\n      addProviderTiles(\"Esri.NatGeoWorldMap\") %>%\n"

# Add markers for each project
for _, row in projects_df.iterrows():
    icon_name = row['Icon'].split('.')[0]
    server_function_script += marker_template.format(
        lng=row['Longitude'],
        lat=row['Latitude'],
        popup=row['Description'].replace('"', '\\"'),  # Escape double quotes in the description
        icon_name=icon_name
    )

# Close the leaflet render function and the server function
server_function_script += "  })\n}"

# Output the generated script for copy/paste
print(server_function_script)


server <- function(input, output, session) {

  COTOIcon <- makeIcon(
    iconUrl = "../images/icons/COTO.png",
    iconWidth = 50, iconHeight = 50,
    iconAnchorX = 25, iconAnchorY = 50
  )

  blandingsIcon <- makeIcon(
    iconUrl = "../images/icons/blandings.png",
    iconWidth = 50, iconHeight = 50,
    iconAnchorX = 25, iconAnchorY = 50
  )

  GalapTortIcon <- makeIcon(
    iconUrl = "../images/icons/GalapTort.png",
    iconWidth = 50, iconHeight = 50,
    iconAnchorX = 25, iconAnchorY = 50
  )

  output$map <- renderLeaflet({
    leaflet() %>%
      addProviderTiles("Esri.NatGeoWorldMap") %>%

addMarkers(
  lng = -105.870087, 
  lat = 34.519939, 
  popup = "We evaluated the ecological significance of abandoned mines in New Mexico as habitats for bats to recommend them for closure or federal protection. This included the collection of habitat, LiDAR imagery, and disease metrics. Field techniques involved off-roading, hiking, and ropework, with additional avian surveys at mine clo